In [1]:
import pandas as pd
import os
import pyarrow
import openpyxl


In [17]:
from dataclasses import dataclass
import numpy as np

resample_rule = '5min'


def resampler_by_treshold(x, treshold):
    # здесь может быть ваш код для ресемплирования
    max_elem = np.amax(x)
    if max_elem > treshold:
        return max_elem
    else:
        return x.mean()

def positive(x):
    val = x.mean()
    return val if val >= 0 else 0.0


def default_resampler(x):
    return resampler_by_treshold(x, 100000)



def resampler_current_startor(x):
    return resampler_by_treshold(x, 1000)

def resampler_vibroplatform4_prodol(x):
    return resampler_by_treshold(x, 10)

def resampler_vibroplatform4_prodol(x):
    return resampler_by_treshold(x, 10)

def resampler_vibroplatform3(x):
    return resampler_by_treshold(x, 7)


@dataclass
class TrainData:
    all: pd.DataFrame
    filtered_na_marked_ffill: pd.DataFrame
    filtered_na_marked_mean: pd.DataFrame
    filtered_resampled_na_ffill: pd.DataFrame
    filtered_resampled_na_mean: pd.DataFrame
    filtered_resampled_na_ffill_to_mean: pd.DataFrame

    @classmethod
    def construct_from(cls, all, filtered):
        na_marked = filtered.fillna(-5)
        na_ffilled = filtered.fillna(method='ffill')
        na_ffilled_rolling_window = filtered.fillna(filtered.rolling(2, min_periods=1).mean())

        columns = na_ffilled_rolling_window.columns
        columns_names = {
            i: name for i, name in enumerate(na_ffilled_rolling_window.columns)
        }
        apply_mapping = {
            i: 'mean' for i in range(16)
        }
        # apply_mapping[2] = resampler_current_startor
        apply_mapping[12] = resampler_vibroplatform3
        apply_mapping[15] = resampler_vibroplatform4_prodol

        transformed_mappings = {
           columns_names[idx]: mapping for (idx, mapping) in apply_mapping.items()
        }
        # print(columns_names[2])
        print(columns_names[12])
        print(columns_names[15])

        return TrainData(
            all=all,
            filtered_na_marked_ffill=na_marked.resample(resample_rule).ffill(),
            filtered_na_marked_mean=na_marked.resample(resample_rule).mean(),
            filtered_resampled_na_ffill=na_ffilled.resample(resample_rule).ffill(),
            # filtered_resampled_na_mean=na_ffilled_rolling_window.resample(resample_rule).mean(),
            filtered_resampled_na_mean=na_ffilled_rolling_window.resample(resample_rule).mean(),
            filtered_resampled_na_ffill_to_mean=na_ffilled.resample(resample_rule).mean()
        )






In [3]:
train_or_test = 'train' if True else 'test'
x_all = pd.read_parquet(os.path.join('..', 'Датасеты', f'X_{train_or_test}.parquet'), engine='pyarrow')
x_filtered = x_all[[x for x in x_all.columns if f'ЭКСГАУСТЕР {4}' in x ]]




In [31]:

val = TrainData.construct_from(x_all, x_filtered)



ЭКСГАУСТЕР 4. ТОК СТАТОРА
ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 3
ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.


In [ ]:

plot_x(val.filtered_resampled_na_mean,     f'exg{4}_train_filtered_resampled_na_mean')


In [4]:
import matplotlib.pyplot as plt


def plot_x(X, filename):
    fig, axs = plt.subplots(len(X.columns), 1, figsize=(20, 100))
    for i, head in enumerate(X.columns):
        ax = axs[i]
        ax.plot(X.index, X[head], label=head)
        ax.set_xlim(X.index[0], X.index[-1])
        ax.axhline(y=0, color='red')
        ax.legend()

    plt.plot()
    plt.savefig(filename + '.png', dpi=300)
    plt.clf()


def plot_y(y, filename):
    fig, axs = plt.subplots(len(y.columns), 1, figsize=(20, 20))

    for j, head in enumerate(y.columns):
        ax = axs[j]
        ax.plot(y.index, y[head], label=head)
        ax.set_xlim(y.index[0], y.index[-1])
        ax.axhline(y=0, color='red')
        ax.legend()
    plt.savefig(gilename + '.png', dpi=300)
    plt.clf()


In [13]:



train_x_data = load_x_data_for_exgauster(exgauster_num=4, is_train=True)



KeyError: 'Column(s) [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15] do not exist'

In [ ]:

plot_x(train_x_data.filtered_na_marked_mean,     f'exg{4}_train_filtered_na_marked_mean')



In [19]:
# for i in [4, 5, 6, 7, 8, 9]:
for i in [4, 5, 6, 7, 8, 9]:
    train_x_data = load_x_data_for_exgauster(exgauster_num=i, is_train=True)
    test_x_data = load_x_data_for_exgauster(exgauster_num=i, is_train=False)
     # train_y_data = load_y_data_for_exgauster(exgauster_num=i)

    plot_x(train_x_data.filtered_na_marked_mean,     f'exg{i}_train_filtered_na_marked_mean')
    plot_x(train_x_data.filtered_na_marked_ffill,    f'exg{i}_train_filtered_na_marked_ffil')
    plot_x(train_x_data.filtered_resampled_na_ffill, f'exg{i}_train_filtered_resampled_na_ffill')
    plot_x(train_x_data.filtered_resampled_na_mean,  f'exg{i}_train_filtered_resampled_na_mean')
    plot_x(train_x_data.filtered_resampled_na_ffill_to_mean,  f'exg{i}_train_filtered_resampled_na_ffill_to_mean')

    plot_x(test_x_data.filtered_na_marked_mean,      f'exg{i}_test_filtered_na_marked_mean')
    print(f'exg{i}_test_filtered_na_marked_ffil')
    plot_x(test_x_data.filtered_na_marked_ffill,     f'exg{i}_test_filtered_na_marked_ffil')
    print(f'exg{i}_test_filtered_na_marked_mean')
    plot_x(test_x_data.filtered_na_marked_mean,     f'exg{i}_test_filtered_na_marked_mean')
    print(f'exg{i}_test_filtered_resampled_na_ffill')
    plot_x(test_x_data.filtered_resampled_na_ffill,  f'exg{i}_test_filtered_resampled_na_ffill')
    print(f'exg{i}_test_filtered_resampled_na_mean')
    plot_x(test_x_data.filtered_resampled_na_mean,   f'exg{i}_test_filtered_resampled_na_mean')
    print(f'exg{i}_test_filtered_resampled_na_ffill_to_mean')
    plot_x(test_x_data.filtered_resampled_na_ffill_to_mean,   f'exg{i}_test_filtered_resampled_na_ffill_to_mean')

    # plot_x(train_y_data.filtered_na_marked_mean,     f'exg{i}_y_train_filtered_na_marked_mean')







ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 3
ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 3
ЭКСГАУСТЕР 4. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
exg4_test_filtered_na_marked_ffil
exg4_test_filtered_na_marked_mean
exg4_test_filtered_resampled_na_ffill
exg4_test_filtered_resampled_na_mean
exg4_test_filtered_resampled_na_ffill_to_mean
ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 3
ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 3
ЭКСГАУСТЕР 5. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
exg5_test_filtered_na_marked_ffil
exg5_test_filtered_na_marked_mean
exg5_test_filtered_resampled_na_ffill
exg5_test_filtered_resampled_na_mean


C:\Users\artem\AppData\Local\Temp\ipykernel_30920\4157132090.py:5: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(len(X.columns), 1, figsize=(20, 100))


exg5_test_filtered_resampled_na_ffill_to_mean
ЭКСГАУСТЕР 6. ВИБРАЦИЯ НА ОПОРЕ 3
ЭКСГАУСТЕР 6. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
ЭКСГАУСТЕР 6. ВИБРАЦИЯ НА ОПОРЕ 3
ЭКСГАУСТЕР 6. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
exg6_test_filtered_na_marked_ffil
exg6_test_filtered_na_marked_mean
exg6_test_filtered_resampled_na_ffill
exg6_test_filtered_resampled_na_mean
exg6_test_filtered_resampled_na_ffill_to_mean
ЭКСГАУСТЕР 7. ВИБРАЦИЯ НА ОПОРЕ 3
ЭКСГАУСТЕР 7. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
ЭКСГАУСТЕР 7. ВИБРАЦИЯ НА ОПОРЕ 3
ЭКСГАУСТЕР 7. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
exg7_test_filtered_na_marked_ffil
exg7_test_filtered_na_marked_mean
exg7_test_filtered_resampled_na_ffill
exg7_test_filtered_resampled_na_mean
exg7_test_filtered_resampled_na_ffill_to_mean
ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 3
ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 3
ЭКСГАУСТЕР 8. ВИБРАЦИЯ НА ОПОРЕ 4. ПРОДОЛЬНАЯ.
exg8_test_filtered_na_marked_ffil
exg8_test_filtered_na_marked_mean
exg8_test_filtered_res

<Figure size 2000x10000 with 0 Axes>

<Figure size 2000x10000 with 0 Axes>

...

<Figure size 2000x10000 with 0 Axes>

<Figure size 2000x10000 with 0 Axes>

<Figure size 2000x10000 with 0 Axes>

<Figure size 2000x10000 with 0 Axes>

<Figure size 2000x10000 with 0 Axes>